<table align="left">
  <td>
    <a href="https://tinyurl.com/2pkzxsha" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Transfer Learning con TensorFlow

En este notebook vamos a ver cómo podemos usar transfer learning para mejorar el rendimiento de un modelo de clasificación de imágenes.

Transfer learning es una técnica de aprendizaje automático que nos permite aprovechar los patrones aprendidos por un modelo en un problema para resolver otro problema.

Hay dos beneficios principales al usar transfer learning:

- Puede aprovechar una arquitectura de red neuronal existente probada para funcionar en problemas similares a los nuestros.
- Puede aprovechar una arquitectura de red neuronal que ya funciona y que ha aprendido patrones en datos similares a los nuestros. Esto a menudo resulta en lograr excelentes resultados con menos datos personalizados.

:book: **Vamos a ver lo siguiente con TensorFlow:**

- Introdución a transfer learning (una forma de superar todos nuestros modelos anteriores)
- Usar un dataset más pequeño para experimentar más rápido (10% de las muestras de entrenamiento de 10 clases de comida)
- Construir un modelo de extracción de características de transfer learning usando TensorFlow Hub
- Introducir el callback TensorBoard para rastrear los resultados del entrenamiento del modelo
- Comparar los resultados del modelo usando TensorBoard
